In [2]:
!pip install qiskit
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 91.3 MB/s eta 0:00:00


In [4]:
# Modern imports
import random
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_aer import AerSimulator
from typing import List, Tuple

# Fix random seed for reproducible examples (remove or change for real randomness)
random.seed(42)

# Backend simulator
backend = AerSimulator()

In [6]:
def random_bits(n: int) -> List[int]:
    return [random.randint(0, 1) for _ in range(n)]

def random_bases(n: int) -> List[int]:
    """Return 0 for Z-basis (|0>, |1>), 1 for X-basis (|+>, |->)"""
    return [random.randint(0, 1) for _ in range(n)]

def prepare_circuits_for_alice(bits: List[int], bases: List[int]) -> List[QuantumCircuit]:
    """Create one circuit per qubit encoding Alice's preparation."""
    circuits = []
    for b, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        # encode bit in Z basis by applying X if bit==1
        if b == 1:
            qc.x(0)
        # if basis is X, apply H to prepare |+> / |->
        if basis == 1:
            qc.h(0)
        # leave a measurement placeholder for later (Bob or Eve will measure)
        circuits.append(qc)
    return circuits


def measure_in_basis_circuits(bases: List[int], incoming_circuits: List[QuantumCircuit]) -> List[QuantumCircuit]:
    """Given basis choices and existing single-qubit circuits (prepared qubit),
    append measurement operations for Bob (or Eve)."""
    circuits = []
    for basis, base_qc in zip(bases, incoming_circuits):
        qc = base_qc.copy()
        # if measuring in X basis, rotate with H before measuring
        if basis == 1:
            qc.h(0)
        qc.measure(0, 0)
        circuits.append(qc)
    return circuits


def run_circuits_get_bits(circuits: List[QuantumCircuit], shots: int = 1) -> List[int]:
    """Run circuits on AerSimulator in a batch and return list of measured bits (one shot per circuit).
    We assume one classical bit per circuit and shots=1 per circuit.)"""
    if not circuits:
        return []
    # transpile for backend
    t_circuits = transpile(circuits, backend=backend)
    job = backend.run(t_circuits, shots=shots)
    result = job.result()
    counts_list = [result.get_counts(i) for i in range(len(t_circuits))]
    measured = []
    for counts in counts_list:
        # counts is a dict like {'0': 1} or {'1': 1}
        # since shots=1, pick the single key
        bit = int(next(iter(counts)))
        measured.append(bit)
    return measured


def eve_intercept_resend(alice_circuits: List[QuantumCircuit], eve_bases: List[int]) -> List[QuantumCircuit]:
    """Eve measures each incoming qubit in her chosen basis, then re-prepares a fresh qubit in the result and basis she measured (intercept-resend).
    Returns new circuits prepared to be sent to Bob.
    """
    # First, measure Alice's circuits in Eve's bases
    measure_circuits = measure_in_basis_circuits(eve_bases, [qc.copy() for qc in alice_circuits])
    # run these to get measurement outcomes
    eve_measurements = run_circuits_get_bits(measure_circuits)

    # Now re-prepare qubits according to Eve's measurement results and send them onward
    forwarded = []
    for m, basis in zip(eve_measurements, eve_bases):
        qc = QuantumCircuit(1, 1)
        if m == 1:
            qc.x(0)
        if basis == 1:
            qc.h(0)
        # don't measure here (Bob will measure later)
        forwarded.append(qc)
    return forwarded


def sift_key(alice_bits: List[int], alice_bases: List[int], bob_bits: List[int], bob_bases: List[int]) -> Tuple[List[int], List[int], List[int]]:
    """Return (sifted_alice, sifted_bob, indices_kept) where we keep positions with matching bases."""
    sifted_a = []
    sifted_b = []
    indices = []
    for i, (ab, aa, bb) in enumerate(zip(alice_bases, alice_bits, bob_bits)):
        if ab == ab:  # no-op, kept for readability
            pass
        # keep if bases match
        if alice_bases[i] == bob_bases[i]:
            sifted_a.append(alice_bits[i])
            sifted_b.append(bob_bits[i])
            indices.append(i)
    return sifted_a, sifted_b, indices


def error_rate(a_bits: List[int], b_bits: List[int]) -> float:
    if not a_bits:
        return 0.0
    mismatches = sum(x != y for x, y in zip(a_bits, b_bits))
    return mismatches / len(a_bits)


## Full BB84 run function

This function runs the whole protocol for `n` qubits. Set `with_eve=True` to include an intercept-resend Eve. The function returns Alice and Bob's raw bits and bases, plus sifted keys and the estimated error rate.

In [7]:
def run_bb84(n: int = 16, with_eve: bool = False, verbose: bool = True):
    # 1) Alice chooses bits and bases
    alice_bits = random_bits(n)
    alice_bases = random_bases(n)

    # 2) Alice prepares qubits (one circuit per qubit)
    alice_circuits = prepare_circuits_for_alice(alice_bits, alice_bases)

    # 3) (Optional) Eve intercepts and resends
    if with_eve:
        eve_bases = random_bases(n)
        forwarded_circuits = eve_intercept_resend(alice_circuits, eve_bases)
    else:
        # no Eve: forward Alice's prepared circuits unchanged
        forwarded_circuits = [qc.copy() for qc in alice_circuits]

    # 4) Bob chooses measurement bases
    bob_bases = random_bases(n)
    # Append measurement operations for Bob
    bob_measure_circuits = measure_in_basis_circuits(bob_bases, forwarded_circuits)

    # 5) Run Bob's measurements
    bob_bits = run_circuits_get_bits(bob_measure_circuits)

    # 6) Sift keys (Alice and Bob publicly compare bases and keep matching ones)
    sifted_a, sifted_b, indices = sift_key(alice_bits, alice_bases, bob_bits, bob_bases)

    # 7) Estimate error rate by comparing a random sample or the whole sifted key
    err = error_rate(sifted_a, sifted_b)

    if verbose:
        print(f"Alice bits:      {alice_bits}")
        print(f"Alice bases (0=Z,1=X): {alice_bases}")
        print(f"Bob bits:        {bob_bits}")
        print(f"Bob bases (0=Z,1=X):   {bob_bases}")
        print(f"Sifted indices (bases matched): {indices}")
        print(f"Sifted Alice:    {sifted_a}")
        print(f"Sifted Bob:      {sifted_b}")
        print(f"Estimated error rate on sifted key: {err:.3f}")

    return {
        'alice_bits': alice_bits,
        'alice_bases': alice_bases,
        'bob_bits': bob_bits,
        'bob_bases': bob_bases,
        'sifted_alice': sifted_a,
        'sifted_bob': sifted_b,
        'sifted_indices': indices,
        'error_rate': err
    }


## Example runs

Run the protocol without Eve and with Eve and compare error rates.

In [8]:
# Run without Eve
res_no_eve = run_bb84(n=24, with_eve=False)

# Run with Eve (intercept-resend)
res_with_eve = run_bb84(n=24, with_eve=True)

print('\nSummary:')
print(f"No Eve - sifted length: {len(res_no_eve['sifted_alice'])}, error rate: {res_no_eve['error_rate']:.3f}")
print(f"With Eve - sifted length: {len(res_with_eve['sifted_alice'])}, error rate: {res_with_eve['error_rate']:.3f}")

Alice bits:      [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1]
Alice bases (0=Z,1=X): [1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0]
Bob bits:        [1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1]
Bob bases (0=Z,1=X):   [0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1]
Sifted indices (bases matched): [1, 3, 8, 10, 11, 13, 16, 17, 18, 20, 21]
Sifted Alice:    [0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0]
Sifted Bob:      [0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0]
Estimated error rate on sifted key: 0.000
Alice bits:      [1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0]
Alice bases (0=Z,1=X): [0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]
Bob bits:        [0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
Bob bases (0=Z,1=X):   [1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]
Sifted indices (bases matched): [1, 3,

## Notes & Next steps

- This implementation uses single-shot circuits for each qubit and batches them to the Aer simulator. That mirrors the BB84 idea where each qubit is sent once.
- For real experiments consider noise, larger shots, error correction, and privacy amplification.

If you want, I can also:
- Add an explicit sampling step where Alice and Bob publicly compare a subset of the sifted key to estimate error rate (rather than using all bits),
- Vectorize the circuit construction further to reduce overhead, or
- Export the notebook to a `.py` script or a cleaned notebook with richer visualizations.


In [21]:
# OPTIONAL - Using the sifted key for encryption/decryption
import binascii

def encrypt_message(unencrypted_string: str, key: str) -> str:
    # Convert ASCII string to binary string
    bits = bin(int(binascii.hexlify(unencrypted_string.encode("utf-8", "surrogatepass")), 16))[2:]
    bitstring = bits.zfill(8 * ((len(bits) + 7) // 8))
    # XOR with key
    encrypted_string = ""
    for i in range(len(bitstring)):
        encrypted_string += str(int(bitstring[i]) ^ int(key[i]))
    return encrypted_string

def decrypt_message(encrypted_bits: str, key: str) -> str:
    # XOR back with key
    unencrypted_bits = "".join(
        str(int(encrypted_bits[i]) ^ int(key[i]))
        for i in range(len(encrypted_bits))
    )

    # Convert to bytes
    i = int(unencrypted_bits, 2)
    hex_string = "%x" % i
    n = len(hex_string)
    raw_bytes = binascii.unhexlify(hex_string.zfill(n + (n & 1)))

    try:
        # Try decoding as UTF-8
        return raw_bytes.decode("utf-8", "surrogatepass")
    except UnicodeDecodeError:
        # Show corrupted output as hex and ASCII fallback
        hex_out = raw_bytes.hex()
        ascii_fallback = "".join(chr(b) if 32 <= b <= 126 else "?" for b in raw_bytes)
        return f"[Corrupted] HEX={hex_out[:64]}...  ASCII≈'{ascii_fallback[:32]}...'"

def demonstrate_eve(secret_message="Quantum Computing is cool :)"):
    print("=== Without Eve ===")
    res_no_eve = run_bb84(n=64, with_eve=False, verbose=False)

    alice_key = "".join(str(b) for b in res_no_eve["sifted_alice"])
    bob_key   = "".join(str(b) for b in res_no_eve["sifted_bob"])

    needed_len = len(secret_message) * 8
    key_for_alice = (alice_key * ((needed_len // len(alice_key)) + 1))[:needed_len]
    key_for_bob   = (bob_key   * ((needed_len // len(bob_key)) + 1))[:needed_len]

    encrypted = encrypt_message(secret_message, key_for_alice)
    decrypted = decrypt_message(encrypted, key_for_bob)

    print("Error rate:", res_no_eve["error_rate"])
    print("Original:", secret_message)
    print("Encrypted:", encrypted)
    print("Decrypted by Bob:", decrypted)
    print()

    print("=== With Eve (intercept-resend) ===")
    res_with_eve = run_bb84(n=64, with_eve=True, verbose=False)

    alice_key = "".join(str(b) for b in res_with_eve["sifted_alice"])
    bob_key   = "".join(str(b) for b in res_with_eve["sifted_bob"])

    needed_len = len(secret_message) * 8
    key_for_alice = (alice_key * ((needed_len // len(alice_key)) + 1))[:needed_len]
    key_for_bob   = (bob_key   * ((needed_len // len(bob_key)) + 1))[:needed_len]

    encrypted = encrypt_message(secret_message, key_for_alice)
    decrypted = decrypt_message(encrypted, key_for_bob)

    print("Error rate:", res_with_eve["error_rate"])
    print("Original:", secret_message)
    print("Encrypted:", encrypted)
    print("Decrypted by Bob:", decrypted)


# Run the demonstration
demonstrate_eve()

=== Without Eve ===
Error rate: 0.0
Original: Quantum Computing is cool :)
Decrypted by Bob: Quantum Computing is cool :)

=== With Eve (intercept-resend) ===
Error rate: 0.21621621621621623
Original: Quantum Computing is cool :)
Decrypted by Bob: [Corrupted] HEX=5d7551367c156ca2832c6d7c6376716e079079b32366efe96c38162d...  ASCII≈']uQ6|?l??,m|cvqn??y?#f??l8?-...'
